In [ ]:
# Dataset: Cómo se lee de disco (SSD o HDD) los datos y se carga a RAM. (lee uno por uno)
# Transforms: Suposiciones sobre el data generating distribution
# DataLoader: Agrupar las imágenes en batches
# Suponemos que estamos haciendo Supervised Learning (inputs (x), targets(y))
#     - p(y=y|x=x)
# Training loop: 
#     - Definir la loss function (mide la dis-similitud entre las dist. out y target)
#     - Forward pass: Dado un input (batch), predecir el output (batch)
#     - Calcular el valor de la loss function: usando el output y target
#     - Calcular la gradiente del valor de la loss function (pytorch: autograd)
#     - Optimizer:
#         - Adam (momentum), otros
#         - optimizer = Adam(model.parameters(), lr=..., momentum=...)
#         - optimizer.step() (después de haber calc las gradientes)
#             - Step: wf = wi - lr * g
#     - Learning Rate Scheduler: Cómo varia el learning rate durante el entrenamiento

In [ ]:
PATH = "../input/plant-pathology-2020-fgvc7/"
IMGS_PATH = PATH + "images/"
TRAIN_CSV = PATH + "train.csv"
TEST_CSV = PATH + "test.csv"

In [ ]:
import math

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.io import read_image
import torchvision.transforms as T
import torchvision.transforms.functional as F
import torch.nn as nn
import torch.optim as optim

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
from fastai.vision.models import resnet18
from fastai.vision.data import imagenet_stats

In [ ]:
class CFG:
    num_classes = -1
    img_size = 480
    seed = 2020
    batch_size = 16
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Dataframe
df = pd.read_csv("../input/plant-pathology-2020-fgvc7/train.csv")
df.head()

In [ ]:
df.shape

In [ ]:
CFG.num_classes = df.shape[1] - 1
CFG.num_classes

In [ ]:
class PlantPathology2020(Dataset): # input (img Train_0.jpg), target (labels i.e. 0 0 0 1)
    def __init__(self, csv_file, imgs_path, item_tfms=None):
        self.df = pd.read_csv(csv_file) # Dataframe
        self.imgs_path = imgs_path # dir de las imagenes
        self.item_tfms = item_tfms # Resize de las imágenes
        self.len = self.df.shape[0] # n ejemplos
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx): # devolver un par (input (img), target (vec: one-hot encoded))
        row = self.df.iloc[idx]
        image_path = self.imgs_path + row[0] + '.jpg'
        #  input (torch.tensor) -> model -> output (torch.tensor)
        image = read_image(input_path) # 3x1365x2048 o 3x2048x1365; 0 1 2; -3 -2 -1
        target = torch.tensor(row[-4:], dtype=torch.float32)
        if self.item_tfms:
            return self.item_tfms(image), target
        return image, target

In [ ]:
class CenterCrop(object):
    def __init__(self, output_size: int, padding_mode: str="symmetric"):
        self.padding_mode = padding_mode
        self.center_crop = T.CenterCrop(output_size)
    
    def __call__(self, x):
        if x.shape[-2] > x.shape[-1]:
            x = x.transpose(-1, -2)
        d1, d2 = math.floor((x.shape[-1] - x.shape[-2]) / 2), math.ceil((x.shape[-1] - x.shape[-2]) / 2)
        x = F.pad(x, padding=[0, d1, 0, d2], padding_mode=self.padding_mode)
        # asumimos que las dimensiones son mayores que el output_size
        return self.center_crop(x)

In [ ]:
# Transforms (con Transfer Learning: usar una red pre-entrenada en Imagenet por ejemplo) mean, std
# Resize en CPU
item_tfms = CenterCrop(CFG.img_size)
# Traininig set transforms (misc, luego normalizar) en GPU
batch_tfms = T.Compose([
    T.RandomHorizontalFlip(p=0.2),
    T.RandomVerticalFlip(p=0.2),
    T.RandomApply([T.ColorJitter(brightness=0.10, contrast=0.10, saturation=0.10, hue=0.10)], p=0.2),
    T.RandomApply(T.GaussianBlur(3), p=0.05),
    T.Normalize(*imagenet_stats)
])
# Validation set transforms (solo normalizar) en GPU
valid_tfms = T.Normalize(*imagenet_stats)

In [ ]:
#Instanciar dataset
dataset = PlantPathology2020(csv_file=TRAIN_CSV,
                            imgs_path=IMGS_PATH,
                            item_tfms=item_tfms)

In [ ]:
train_len = math.floor(0.8 * len(dataset))
valid_len = len(dataset) - train_len
train_len, valid_len

In [ ]:
# DataLoaders 80/20
train_set, valid_set = random_split(dataset, [train_len, valid_len], generator=torch.Generator().manual_seed(CFG.seed))
train_loader = DataLoader(train_set,
                          batch_size=CFG.batch_size,
                          shuffle=True,
                          pin_memory=True)
valid_loader = DataLoader(valid_set,
                          batch_size=CFG.batch_size,
                          pin_memory=True)

In [ ]:
model = resnet18()
model.to(CFG.device)

In [ ]:
model.fc.in_features

In [ ]:
model.fc = nn.Linear(model.fc.in_features, CFG.num)

In [ ]:
t = torch.tensor([[1, 2, 3, 12, 1, 20, 5],
                  [1, 2, 30, -12, 1, -20, 18],
                  [1, -2, 3, 12, -11, 35, 8]]).float()
t

In [ ]:
torch.softmax(t, dim=1)

In [ ]:
# cuando se hace toda una pasada en el dataset, ha pasado un epoch
epochs = 10
criterion = nn.CrossEntropyLoss() # recibe output del modelo (logits), target

In [ ]:
y = torch.tensor([[0, 0, 0, 1, 0],
                 [0, 0, 1, 0, 0],
                 [0, 1, 0, 0, 0]])
y

In [ ]:
y_pred = torch.rand(y.shape)
y_pred

In [ ]:
loss = torch.tensor(5)
loss.item()

In [ ]:
training_loss = 100

In [ ]:
training_loss += loss

In [ ]:
training_loss

In [ ]:
y.argmax(dim=1)

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
def train():
    #Training
    for e in range(epochs):
        train_loss = 0
        model.train()
        for x, y in train_loader:
            x, y = x.cuda(non_blocking=True), y.cuda(non_blocking=True)
            x = batch_tfms(x)
            y_pred = model(x)
            loss = criterion(y_pred, y.argmax(dim=1))
            optimizer.zero_grad() # hace que las gradientes calculadas en la iteración anterior sean 0
            loss.backward() # calcula las gradientes
            train_loss += loss.item() 
            optimizer.step() # actualiza los pesos de los tensores de las capas de la red
        model.eval()
        valid_loss = 0
        with torch.no_grad():
            for x, y in valid_loader:
                x, y = x.cuda(non_blocking=True), y.cuda(non_blocking=True)
                x = valid_tfms(x)
                y_pred = model(x)
                loss = criterion(y_pred, y.argmax(dim=1))
                valid_loss += loss.item()
        print(f"{e} training loss:", train_loss / len(train_loader))
        print(f"{e} valid loss:", valid_loss / len(valid_loader))